In [4]:
import os
os.name

'posix'

In [ ]:
# TODO - Orient analysis grid to longest edge of zone it's being generated for
# TODO - Add radiance surface properties to HBSurfaces based on IDF properties/corresponding materials
# TODO - Create ability to visualise surfaces and analsyis grids in context
# TODO - Add radiance parameters to the config.json

# Load necessary packages
import sys
sys.path.insert(0,'ladybug')
sys.path.insert(0,'honeybee')
from honeybee.hbsurface import HBSurface
from honeybee.hbfensurface import HBFenSurface
from honeybee.radiance.analysisgrid import AnalysisGrid
from honeybee.radiance.properties import RadianceProperties
from honeybee.radiance.material.glass import Glass
from honeybee.radiance.material.plastic import Plastic
import matplotlib.patches as patches
from honeybee.radiance.analysisgrid import AnalysisGrid
from honeybee.radiance.sky.skymatrix import SkyMatrix
from honeybee.radiance.recipe.daylightfactor.gridbased import GridBased as df_GridBased
from honeybee.radiance.recipe.annual.gridbased import GridBased as annual_GridBased

import eppy
from eppy import modeleditor
from eppy.modeleditor import IDF
import json

def loadJSON(path):
    """
    Description:
        Load a JSON file into a dictionary object
    Arguments:
        path [string]: The location of the JSON file being loaded
    Returns:
        dictionary [dict]: Dictionary containing contents of loaded JSON file 
    """
    import json
    with open(path) as data_file:
        return json.load(data_file)

# Specify the config file to be referenced
config = loadJSON("idf_config.json")

# Run the LoadModifyIDF.py script
#import subprocess
#subprocess.call(["python", "LoadModifyIDF.py", idf_config_file])

idf_file = config["target_idf"]
idd_file = config["idd_file"]
epw_file = config["weather_file"]
IDF.setiddname(idd_file)
idf = IDF(idf_file)

# Set the "vector to north", so that wall orientation can be obtained
north_angle = np.radians(idf.idfobjects["BUILDING"][0].North_Axis)
north_vector = (np.sin(north_angle), np.cos(north_angle), 0)
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

def rotate(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox, oy = origin
    px, py = point

    qx = ox + np.cos(angle) * (px - ox) - np.sin(angle) * (py - oy)
    qy = oy + np.sin(angle) * (px - ox) + np.cos(angle) * (py - oy)
    return qx, qy

# List the zones to be analysed - I'm thinking this will come from GH initially
zones_to_analyse = config["analysis_zones"]

# Define materials to be applied to surfaces
glass_material_N = Glass("GlassMaterialN", r_transmittance=config["glazing_visible_transmittance_N"], g_transmittance=config["glazing_visible_transmittance_N"], b_transmittance=config["glazing_visible_transmittance_N"], refraction_index=1.52)
glass_material_S = Glass("GlassMaterialS", r_transmittance=config["glazing_visible_transmittance_S"], g_transmittance=config["glazing_visible_transmittance_S"], b_transmittance=config["glazing_visible_transmittance_S"], refraction_index=1.52)
glass_material_E = Glass("GlassMaterialE", r_transmittance=config["glazing_visible_transmittance_E"], g_transmittance=config["glazing_visible_transmittance_E"], b_transmittance=config["glazing_visible_transmittance_E"], refraction_index=1.52)
glass_material_W = Glass("GlassMaterialW", r_transmittance=config["glazing_visible_transmittance_W"], g_transmittance=config["glazing_visible_transmittance_W"], b_transmittance=config["glazing_visible_transmittance_W"], refraction_index=1.52)
glass_material_internal = Glass("GlassMaterialInternal", r_transmittance=0.9, g_transmittance=0.9, b_transmittance=0.9, refraction_index=1.52)
glass_material_skylight = Glass("GlassMaterialSkylight", r_transmittance=config["glazing_visible_transmittance_skylight"], g_transmittance=config["glazing_visible_transmittance_skylight"], b_transmittance=config["glazing_visible_transmittance_skylight"], refraction_index=1.52)

air_wall_material = Glass("AirWallMaterial", r_transmittance=0, g_transmittance=0, b_transmittance=0, refraction_index=1)
wall_material = Plastic("WallMaterial", r_reflectance=config["wall_reflectivity"], g_reflectance=config["wall_reflectivity"], b_reflectance=config["wall_reflectivity"], specularity=0, roughness=0)
ceiling_material = Plastic("CeilingMaterial", r_reflectance=config["ceiling_reflectivity"], g_reflectance=config["ceiling_reflectivity"], b_reflectance=config["ceiling_reflectivity"], specularity=0, roughness=0)
floor_material = Plastic("FloorMaterial", r_reflectance=config["floor_reflectivity"], g_reflectance=config["floor_reflectivity"], b_reflectance=config["floor_reflectivity"], specularity=0, roughness=0)

# Assign surfaces within IDF to HBSurfaces for daylight analysis
exterior_wall_surfaces = []
for wall_n, wall in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if (i.Construction_Name == "Exterior Wall")]):
    srf = HBSurface("wall_{0:}_{1:}".format(wall_n, wall.Name), wall.coords, surface_type=0, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=wall_material))
    angle_to_north = np.degrees(angle_between(north_vector, srf.normal))
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if wall.Name in fenestration.Name:
            # Assign glazing properties based on orientation
            if (angle_to_north > 315) and (angle_to_north <= 45):
                orientation_glazing_material = glass_material_N
            elif (angle_to_north > 45) and (angle_to_north <= 135):
                orientation_glazing_material = glass_material_E
            elif (angle_to_north > 135) and (angle_to_north <= 225):
                orientation_glazing_material = glass_material_S
            elif (angle_to_north > 225) and (angle_to_north <= 315):
                orientation_glazing_material = glass_material_W
            fensrf = HBFenSurface("wall_{0:}_{1:}_fenestration_{2:}_{3:}".format(wall_n, wall.Name, fenestration_n, fenestration.Name), fenestration.coords, rad_properties=RadianceProperties(material=orientation_glazing_material))
            srf.add_fenestration_surface(fensrf)
    exterior_wall_surfaces.append(srf)

interior_wall_surfaces = []
for wall_n, wall in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if (i.Construction_Name == "Interior Wall")]):
    srf = HBSurface("wall_{0:}_{1:}".format(wall_n, wall.Name), wall.coords, surface_type=0, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=wall_material))
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if wall.Name in fenestration.Name:
            fensrf = HBFenSurface("wall_{0:}_{1:}_fenestration_{2:}_{3:}".format(wall_n, wall.Name, fenestration_n, fenestration.Name), fenestration.coords, rad_properties=RadianceProperties(material=glass_material_internal))
            srf.add_fenestration_surface(fensrf)
    interior_wall_surfaces.append(srf)

airwall_surfaces = []
for airwall_n, airwall in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if i.Construction_Name == "Air Wall"]):
    srf = HBSurface("airwall_{0:}_{1:}".format(airwall_n, airwall.Name), airwall.coords, surface_type=4, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=air_wall_material))
    airwall_surfaces.append(srf)

floor_surfaces = []
for floor_n, floor in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if (i.Construction_Name == "Interior Floor") or (i.Construction_Name == "Exterior Floor") or (i.Construction_Name == "Exposed Floor")]):
    srf = HBSurface("floor_{0:}_{1:}".format(floor_n, floor.Name), floor.coords, surface_type=2, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=floor_material))
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if floor.Name in fenestration.Name:
            fensrf = HBFenSurface("floor_{0:}_{1:}_fenestration_{2:}_{3:}".format(floor_n, floor.Name, fenestration_n, fenestration.Name), fenestration.coords, rad_properties=RadianceProperties(material=glass_material_interior))
            srf.add_fenestration_surface(fensrf)
    floor_surfaces.append(srf)
    
ceiling_surfaces = []
for ceiling_n, ceiling in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if (i.Construction_Name == "Interior Ceiling") or (i.Construction_Name == "Exterior Ceiling") or (i.Construction_Name == "Roof")]):
    srf = HBSurface("ceiling_{0:}_{1:}".format(ceiling_n, ceiling.Name), ceiling.coords, surface_type=3, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=ceiling_material))
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if ceiling.Name in fenestration.Name:
            fensrf = HBFenSurface("ceiling_{0:}_{1:}_fenestration_{2:}_{3:}".format(ceiling_n, ceiling.Name, fenestration_n, fenestration.Name), fenestration.coords, rad_properties=RadianceProperties(material=glass_material_skylight))
            srf.add_fenestration_surface(fensrf)
    ceiling_surfaces.append(srf)

context_surfaces = []
for context_n, context in enumerate([i for i in idf.idfobjects["SHADING:BUILDING:DETAILED"]]):
    srf = HBSurface("context_{0:}_{1:}".format(context_n, context.Name), context.coords, surface_type=6, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=wall_material))
    context_surfaces.append(srf)

# Define analysis grids for each zone for simulation in Radiance
zone_grids = []
for zone in zones_to_analyse:
    for floor_srf in idf.idfobjects["BUILDINGSURFACE:DETAILED"]:
        if ("Floor" in floor_srf.Construction_Name) and (zone in floor_srf.Zone_Name):
            vert_xs, vert_ys, vert_zs = list(zip(*floor_srf.coords))
            patch = patches.Polygon(list(zip(*[vert_xs, vert_ys])))
            min_x, max_x, min_y, max_y, max_z = min(vert_xs), max(vert_xs), min(vert_ys), max(vert_ys), max(vert_zs)
            x_range = max_x - min_x
            y_range = max_y - min_y
            g = np.meshgrid(
                np.arange(min_x - (x_range / 2), max_x + (x_range / 2), config["daylight_analysis_grid_spacing"]), 
                np.arange(min_y - (y_range / 2), max_y + (y_range / 2), config["daylight_analysis_grid_spacing"])
            )
            coords = list(zip(*(c.flat for c in g)))
            analysispoints = np.vstack([p for p in coords if patch.contains_point(p, radius=config["daylight_analysis_grid_edge_offset"])])
            grid_points = list(zip(*[np.array(list(zip(*analysispoints)))[0], np.array(list(zip(*analysispoints)))[1], np.repeat(max_z+config["daylight_analysis_grid_surface_offset"], len(analysispoints))]))
            zone_grids.append(AnalysisGrid.from_points_and_vectors(grid_points, name=zone))

# Generate sky matrix for annual analysis
sky_matrix = SkyMatrix.from_epw_file(epw_file, sky_density=2, north=0, hoys=range(0, 8760), mode=0, suffix="")

# Generate a recipe for the DF and ANNUAL simulations (for all the rooms at once)
df_recipe = df_GridBased(analysis_grids=zone_grids, hb_objects=np.concatenate([exterior_wall_surfaces, interior_wall_surfaces, floor_surfaces, ceiling_surfaces, airwall_surfaces, context_surfaces]).tolist())
annual_recipe = annual_GridBased(sky_matrix, analysis_grids=zone_grids, hb_objects=np.concatenate([exterior_wall_surfaces, interior_wall_surfaces, floor_surfaces, ceiling_surfaces, airwall_surfaces, context_surfaces]).tolist()) # <<<<<< add modified raytrace parametrs

# Save the analysis recipe to file for later processing
df_recipe.write(config["output_directory"], project_name="DONOTRUN_ALLROOMS_DF")
annual_recipe.write(config["output_directory"], project_name="DONOTRUN_ALLROOMS_ANNUAL")

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np

# Visualise the whole model!
fig = plt.figure(figsize=(15, 7.5))
ax = Axes3D(fig)
xs, ys, zs = list(zip(*np.array([item for sublist in list(zip(*[i.coords for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"]])) for item in sublist])))
ax.set_xlim([min(xs), max(xs)])
ax.set_ylim([min(ys), max(ys)])
ax.set_zlim([min(zs), max(zs)])
[ax.add_collection3d(Poly3DCollection([j], facecolor=(0, 0, 1, 0.1), edgecolor="w", zorder=0)) for j in [i.coords for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"]]]
[ax.add_collection3d(Poly3DCollection([j], facecolor=(1, 0, 0, 0.5), edgecolor="k", linewidth=1, zorder=-1)) for j in [i.coords for i in idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]]]
try:
    [ax.add_collection3d(Poly3DCollection([j], facecolor=(1, 1, 1, 0.5), edgecolor="k", linewidth=1, zorder=2)) for j in [i.coords for i in idf.idfobjects["SHADING:BUILDING:DETAILED"]]]
except:
    pass

def axisEqual3D(ax):
    extents = np.array([getattr(ax, 'get_{}lim'.format(dim))() for dim in 'xyz'])
    sz = extents[:,1] - extents[:,0]
    centers = np.mean(extents, axis=1)
    maxsize = max(abs(sz))
    r = maxsize/2
    for ctr, dim in zip(centers, 'xyz'):
        getattr(ax, 'set_{}lim'.format(dim))(ctr - r, ctr + r)

axisEqual3D(ax)

In [ ]:
# TODO - Orient analysis grid to longest edge of zone it's being generated for
# TODO - Add radiance surface properties to HBSurfaces based on IDF properties/corresponding materials
# TODO - Create ability to visualise surfaces and analsyis grids in context
# TODO - Add radiance parameters to the config.json

# Load necessary packages
import sys
sys.path.insert(0,'ladybug')
sys.path.insert(0,'honeybee')
from honeybee.hbsurface import HBSurface
from honeybee.hbfensurface import HBFenSurface
from honeybee.radiance.analysisgrid import AnalysisGrid
from honeybee.radiance.properties import RadianceProperties
from honeybee.radiance.material.glass import Glass
from honeybee.radiance.material.plastic import Plastic
import matplotlib.patches as patches
from honeybee.radiance.analysisgrid import AnalysisGrid
from honeybee.radiance.sky.skymatrix import SkyMatrix
from honeybee.radiance.recipe.daylightfactor.gridbased import GridBased as df_GridBased
from honeybee.radiance.recipe.annual.gridbased import GridBased as annual_GridBased

import eppy
from eppy import modeleditor
from eppy.modeleditor import IDF
import json

def loadJSON(path):
    """
    Description:
        Load a JSON file into a dictionary object
    Arguments:
        path [string]: The location of the JSON file being loaded
    Returns:
        dictionary [dict]: Dictionary containing contents of loaded JSON file 
    """
    import json
    with open(path) as data_file:
        return json.load(data_file)

# Specify the config file to be referenced
config = loadJSON("idf_config.json")

# Run the LoadModifyIDF.py script
#import subprocess
#subprocess.call(["python", "LoadModifyIDF.py", idf_config_file])

idf_file = config["target_idf"]
idd_file = config["idd_file"]
epw_file = config["weather_file"]
IDF.setiddname(idd_file)
idf = IDF(idf_file)

# Set the "vector to north", so that wall orientation can be obtained
north_angle = np.radians(idf.idfobjects["BUILDING"][0].North_Axis)
north_vector = (np.sin(north_angle), np.cos(north_angle), 0)
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

def rotate(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox, oy = origin
    px, py = point

    qx = ox + np.cos(angle) * (px - ox) - np.sin(angle) * (py - oy)
    qy = oy + np.sin(angle) * (px - ox) + np.cos(angle) * (py - oy)
    return qx, qy

# List the zones to be analysed - I'm thinking this will come from GH initially
zones_to_analyse = config["analysis_zones"]

# Define materials to be applied to surfaces
glass_material_N = Glass("GlassMaterialN", r_transmittance=config["glazing_visible_transmittance_N"], g_transmittance=config["glazing_visible_transmittance_N"], b_transmittance=config["glazing_visible_transmittance_N"], refraction_index=1.52)
glass_material_S = Glass("GlassMaterialS", r_transmittance=config["glazing_visible_transmittance_S"], g_transmittance=config["glazing_visible_transmittance_S"], b_transmittance=config["glazing_visible_transmittance_S"], refraction_index=1.52)
glass_material_E = Glass("GlassMaterialE", r_transmittance=config["glazing_visible_transmittance_E"], g_transmittance=config["glazing_visible_transmittance_E"], b_transmittance=config["glazing_visible_transmittance_E"], refraction_index=1.52)
glass_material_W = Glass("GlassMaterialW", r_transmittance=config["glazing_visible_transmittance_W"], g_transmittance=config["glazing_visible_transmittance_W"], b_transmittance=config["glazing_visible_transmittance_W"], refraction_index=1.52)
glass_material_internal = Glass("GlassMaterialInternal", r_transmittance=0.9, g_transmittance=0.9, b_transmittance=0.9, refraction_index=1.52)
glass_material_skylight = Glass("GlassMaterialSkylight", r_transmittance=config["glazing_visible_transmittance_skylight"], g_transmittance=config["glazing_visible_transmittance_skylight"], b_transmittance=config["glazing_visible_transmittance_skylight"], refraction_index=1.52)

air_wall_material = Glass("AirWallMaterial", r_transmittance=0, g_transmittance=0, b_transmittance=0, refraction_index=1)
wall_material = Plastic("WallMaterial", r_reflectance=config["wall_reflectivity"], g_reflectance=config["wall_reflectivity"], b_reflectance=config["wall_reflectivity"], specularity=0, roughness=0)
ceiling_material = Plastic("CeilingMaterial", r_reflectance=config["ceiling_reflectivity"], g_reflectance=config["ceiling_reflectivity"], b_reflectance=config["ceiling_reflectivity"], specularity=0, roughness=0)
floor_material = Plastic("FloorMaterial", r_reflectance=config["floor_reflectivity"], g_reflectance=config["floor_reflectivity"], b_reflectance=config["floor_reflectivity"], specularity=0, roughness=0)

# Assign surfaces within IDF to HBSurfaces for daylight analysis
exterior_wall_surfaces = []
for wall_n, wall in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if (i.Construction_Name == "Exterior Wall")]):
    srf = HBSurface("wall_{0:}_{1:}".format(wall_n, wall.Name), wall.coords, surface_type=0, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=wall_material))
    angle_to_north = np.degrees(angle_between(north_vector, srf.normal))
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if wall.Name in fenestration.Name:
            # Assign glazing properties based on orientation
            if (angle_to_north > 315) and (angle_to_north <= 45):
                orientation_glazing_material = glass_material_N
            elif (angle_to_north > 45) and (angle_to_north <= 135):
                orientation_glazing_material = glass_material_E
            elif (angle_to_north > 135) and (angle_to_north <= 225):
                orientation_glazing_material = glass_material_S
            elif (angle_to_north > 225) and (angle_to_north <= 315):
                orientation_glazing_material = glass_material_W
            fensrf = HBFenSurface("wall_{0:}_{1:}_fenestration_{2:}_{3:}".format(wall_n, wall.Name, fenestration_n, fenestration.Name), fenestration.coords, rad_properties=RadianceProperties(material=orientation_glazing_material))
            srf.add_fenestration_surface(fensrf)
    exterior_wall_surfaces.append(srf)

interior_wall_surfaces = []
for wall_n, wall in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if (i.Construction_Name == "Interior Wall")]):
    srf = HBSurface("wall_{0:}_{1:}".format(wall_n, wall.Name), wall.coords, surface_type=0, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=wall_material))
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if wall.Name in fenestration.Name:
            fensrf = HBFenSurface("wall_{0:}_{1:}_fenestration_{2:}_{3:}".format(wall_n, wall.Name, fenestration_n, fenestration.Name), fenestration.coords, rad_properties=RadianceProperties(material=glass_material_internal))
            srf.add_fenestration_surface(fensrf)
    interior_wall_surfaces.append(srf)

airwall_surfaces = []
for airwall_n, airwall in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if i.Construction_Name == "Air Wall"]):
    srf = HBSurface("airwall_{0:}_{1:}".format(airwall_n, airwall.Name), airwall.coords, surface_type=4, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=air_wall_material))
    airwall_surfaces.append(srf)

floor_surfaces = []
for floor_n, floor in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if (i.Construction_Name == "Interior Floor") or (i.Construction_Name == "Exterior Floor") or (i.Construction_Name == "Exposed Floor")]):
    srf = HBSurface("floor_{0:}_{1:}".format(floor_n, floor.Name), floor.coords, surface_type=2, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=floor_material))
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if floor.Name in fenestration.Name:
            fensrf = HBFenSurface("floor_{0:}_{1:}_fenestration_{2:}_{3:}".format(floor_n, floor.Name, fenestration_n, fenestration.Name), fenestration.coords, rad_properties=RadianceProperties(material=glass_material_interior))
            srf.add_fenestration_surface(fensrf)
    floor_surfaces.append(srf)
    
ceiling_surfaces = []
for ceiling_n, ceiling in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if (i.Construction_Name == "Interior Ceiling") or (i.Construction_Name == "Exterior Ceiling") or (i.Construction_Name == "Roof")]):
    srf = HBSurface("ceiling_{0:}_{1:}".format(ceiling_n, ceiling.Name), ceiling.coords, surface_type=3, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=ceiling_material))
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if ceiling.Name in fenestration.Name:
            fensrf = HBFenSurface("ceiling_{0:}_{1:}_fenestration_{2:}_{3:}".format(ceiling_n, ceiling.Name, fenestration_n, fenestration.Name), fenestration.coords, rad_properties=RadianceProperties(material=glass_material_skylight))
            srf.add_fenestration_surface(fensrf)
    ceiling_surfaces.append(srf)

context_surfaces = []
for context_n, context in enumerate([i for i in idf.idfobjects["SHADING:BUILDING:DETAILED"]]):
    srf = HBSurface("context_{0:}_{1:}".format(context_n, context.Name), context.coords, surface_type=6, is_name_set_by_user=True, is_type_set_by_user=True, rad_properties=RadianceProperties(material=wall_material))
    context_surfaces.append(srf)

# Define analysis grids for each zone for simulation in Radiance
zone_grids = []
for zone in zones_to_analyse:
    for floor_srf in idf.idfobjects["BUILDINGSURFACE:DETAILED"]:
        if ("Floor" in floor_srf.Construction_Name) and (zone in floor_srf.Zone_Name):
            vert_xs, vert_ys, vert_zs = list(zip(*floor_srf.coords))
            patch = patches.Polygon(list(zip(*[vert_xs, vert_ys])))
            min_x, max_x, min_y, max_y, max_z = min(vert_xs), max(vert_xs), min(vert_ys), max(vert_ys), max(vert_zs)
            x_range = max_x - min_x
            y_range = max_y - min_y
            g = np.meshgrid(
                np.arange(min_x - (x_range / 2), max_x + (x_range / 2), config["daylight_analysis_grid_spacing"]), 
                np.arange(min_y - (y_range / 2), max_y + (y_range / 2), config["daylight_analysis_grid_spacing"])
            )
            coords = list(zip(*(c.flat for c in g)))
            analysispoints = np.vstack([p for p in coords if patch.contains_point(p, radius=config["daylight_analysis_grid_edge_offset"])])
            grid_points = list(zip(*[np.array(list(zip(*analysispoints)))[0], np.array(list(zip(*analysispoints)))[1], np.repeat(max_z+config["daylight_analysis_grid_surface_offset"], len(analysispoints))]))
            zone_grids.append(AnalysisGrid.from_points_and_vectors(grid_points, name=zone))

# Generate sky matrix for annual analysis
sky_matrix = SkyMatrix.from_epw_file(epw_file, sky_density=2, north=0, hoys=range(0, 8760), mode=0, suffix="")

# Generate a recipe for the DF and ANNUAL simulations (for all the rooms at once)
df_recipe = df_GridBased(analysis_grids=zone_grids, hb_objects=np.concatenate([exterior_wall_surfaces, interior_wall_surfaces, floor_surfaces, ceiling_surfaces, airwall_surfaces, context_surfaces]).tolist())
annual_recipe = annual_GridBased(sky_matrix, analysis_grids=zone_grids, hb_objects=np.concatenate([exterior_wall_surfaces, interior_wall_surfaces, floor_surfaces, ceiling_surfaces, airwall_surfaces, context_surfaces]).tolist()) # <<<<<< add modified raytrace parametrs

# Save the analysis recipe to file for later processing
df_recipe.write(config["output_directory"], project_name="DONOTRUN_ALLROOMS_DF")
annual_recipe.write(config["output_directory"], project_name="DONOTRUN_ALLROOMS_ANNUAL")

In [ ]:

#from honeybee.hbsurface import HBSurface
#from honeybee.hbfensurface import HBFenSurface
#from honeybee.radiance.analysisgrid import AnalysisGrid
#from honeybee.radiance.recipe.daylightfactor.gridbased import GridBased as df_GridBased
#from honeybee.radiance.recipe.annual.gridbased import GridBased as annual_GridBased
#from honeybee.radiance.sky.skymatrix import SkyMatrix

grid_spacing = 0.25
grid_edge_offset = 0.1
grid_surface_offset = 0.85

zones_to_analyse = [str(i.Name) for i in idf.idfobjects["ZONE"] if "Adj" not in i.Name]

# Generate wall surfaces
wall_surfaces = []
for wall_n, wall in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if (i.Construction_Name == "Interior Wall") or (i.Construction_Name == "Exterior Wall")]):
    srf = hbsurface.HBSurface("wall_{0:}_{1:}".format(wall_n, wall.Name), wall.coords, surface_type=0, is_name_set_by_user=True, is_type_set_by_user=True)
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if wall.Name in fenestration.Name:
            fensrf = hbsurface.HBFenSurface("wall_{0:}_{1:}_fenestration_{2:}_{3:}".format(wall_n, wall.Name, fenestration_n, fenestration.Name), fenestration.coords)
            srf.add_fenestration_surface(fensrf)
    wall_surfaces.append(srf)
    
wall_surfaces

# Generate floor surfaces
floor_surfaces = []
for floor_n, floor in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if (i.Construction_Name == "Interior Floor") or (i.Construction_Name == "Exterior Floor") or (i.Construction_Name == "Exposed Floor")]):
    srf = hbsurface.HBSurface("floor_{0:}_{1:}".format(floor_n, floor.Name), floor.coords, surface_type=2, is_name_set_by_user=True, is_type_set_by_user=True)
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if floor.Name in fenestration.Name:
            fensrf = hbsurface.HBFenSurface("floor_{0:}_{1:}_fenestration_{2:}_{3:}".format(floor_n, floor.Name, fenestration_n, fenestration.Name), fenestration.coords)
            srf.add_fenestration_surface(fensrf)
    floor_surfaces.append(srf)
    
floor_surfaces

# Generate ceiling/roof surfaces
ceiling_surfaces = []
for ceiling_n, ceiling in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if (i.Construction_Name == "Interior Ceiling") or (i.Construction_Name == "Exterior Ceiling") or (i.Construction_Name == "Roof")]):
    srf = hbsurface.HBSurface("ceiling_{0:}_{1:}".format(ceiling_n, ceiling.Name), ceiling.coords, surface_type=3, is_name_set_by_user=True, is_type_set_by_user=True)
    for fenestration_n, fenestration in enumerate(idf.idfobjects["FENESTRATIONSURFACE:DETAILED"]):
        if ceiling.Name in fenestration.Name:
            fensrf = hbsurface.HBFenSurface("ceiling_{0:}_{1:}_fenestration_{2:}_{3:}".format(ceiling_n, ceiling.Name, fenestration_n, fenestration.Name), fenestration.coords)
            srf.add_fenestration_surface(fensrf)
    ceiling_surfaces.append(srf)
    
ceiling_surfaces

# Generate air wall surfaces
airwall_surfaces = []
for airwall_n, airwall in enumerate([i for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"] if i.Construction_Name == "Air Wall"]):
    srf = hbsurface.HBSurface("airwall_{0:}_{1:}".format(airwall_n, airwall.Name), airwall.coords, surface_type=4, is_name_set_by_user=True, is_type_set_by_user=True)
    airwall_surfaces.append(srf)
    
airwall_surfaces

# Generate context/shading surfaces
context_surfaces = []
for context_n, context in enumerate([i for i in idf.idfobjects["SHADING:BUILDING:DETAILED"]]):
    srf = hbsurface.HBSurface("context_{0:}_{1:}".format(context_n, context.Name), context.coords, surface_type=6, is_name_set_by_user=True, is_type_set_by_user=True)
    context_surfaces.append(srf)
    
context_surfaces

# Generate analysis grids from the floor surfaces
zone_grids = []
for zone in zones_to_analyse:
    for i in idf.idfobjects["BUILDINGSURFACE:DETAILED"]:
        if ("Floor" in i.Construction_Name) and (zone in i.Zone_Name):

            xs, ys, zs = list(zip(*i.coords))
            patch = patches.Polygon(list(zip(*[xs, ys])))
            min_x, max_x, min_y, max_y, max_z = min(xs), max(xs), min(ys), max(ys), max(zs)
            x_range = max_x - min_x
            y_range = max_y - min_y
            g = np.meshgrid(
                np.arange(min_x - (x_range / 2), max_x + (x_range / 2), grid_spacing), 
                np.arange(min_y - (y_range / 2), max_y + (y_range / 2), grid_spacing)
            )
            coords = list(zip(*(c.flat for c in g)))
            analysispoints = np.vstack([p for p in coords if patch.contains_point(p, radius=grid_edge_offset)])
            grid_points = list(zip(*[np.array(list(zip(*analysispoints)))[0], np.array(list(zip(*analysispoints)))[1], np.repeat(max_z+grid_surface_offset, len(analysispoints))]))

#             fig, ax = plt.subplots(1, 1, figsize=(5, 5))
#             ax.add_artist(patch)
#             ep = np.array(analysispoints)
#             ax.scatter(analysispoints[:, 0], analysispoints[:, 1], s=10, color="k", zorder=5)
#             ax.set_title("# points:{}".format(len(analysispoints)))
#             ax.set_xlim([min_x-0.5, max_x+0.5])
#             ax.set_ylim([min_y-0.5, max_y+0.5])
            zone_grids.append(AnalysisGrid.from_points_and_vectors(grid_points, name=zone))
zone_grids

# Generate sky matrix for annual analysis
sky_matrix = SkyMatrix.from_epw_file(epw_file, sky_density=2, north=0, hoys=range(0, 8760), mode=0, suffix="")

# Generate analysis recipes
df_recipe = df_GridBased(analysis_grids=zone_grids, hb_objects=np.concatenate([wall_surfaces, floor_surfaces, ceiling_surfaces, airwall_surfaces, context_surfaces]).tolist())
annual_recipe = annual_GridBased(sky_matrix, analysis_grids=zone_grids, hb_objects=np.concatenate([wall_surfaces, floor_surfaces, ceiling_surfaces, airwall_surfaces, context_surfaces]).tolist()) # <<<<<< add modified raytrace parametrs

df_bat = df_recipe.write("room", "daylightfactor")
annual_bat = annual_recipe.write("room", "annual")

#df_recipe.run(df_bat, True)

# Save analysis recipes to file for later running

# Run analysis files and save results to disk

# Post-process analysis

In [ ]:
df_recipe.run(df_bat, True)

In [ ]:
with open("room/daylightfactor/daylightfactor/daylightfactor.pts") as f:
    pts = [[float(j) for j in i.strip().split(" ")][0:2] for i in f.readlines()]
pts

with open("room/daylightfactor/daylightfactor/result/daylightfactor.ill", "r") as f:
    values = [100 * (float(i) / 100000) for i in f.readlines()]  
values

fig, ax = plt.subplots(1, 1, figsize=(15, 7.5))
ax.scatter([i[0] for i in pts], [i[1] for i in pts], c=values, cmap="viridis", s=5)
ax.set_xlim([0, 30])
ax.set_ylim([50, 70])
#plt.axis('equal')

In [ ]:
nm=8

# Get length of analysis grid objects
with open("room/daylightfactor/daylightfactor/result/daylightfactor.ill", "r") as f:
    result_vals = [float(i.strip()) for i in f.readlines()][:len([i for i in df_recipe.results()[nm].points])]

fig, ax = plt.subplots(1, 1, figsize=(15, 7.5))
ax.scatter([i.x for i in zone_grids[nm].points], [i.y for i in zone_grids[nm].points], c=result_vals, cmap="viridis", s=30, marker="o")

In [ ]:

#[i for i in zone_grids[0].points]

In [ ]:
# Load SQL table output
import sqlite3
import pandas as pd

conn = sqlite3.connect(r"C:\Users\tgerrish\Desktop\SAMAzure\TestFiles\eplusout.sql")
c = conn.cursor()

print(pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn))

pd.read_sql_query("SELECT * FROM ReportData", conn)

conn.close()

In [ ]:
"C:\EnergyPlusV8-8-0\energyplus.exe" -a -x -r -w "GBR_Cardiff_CIBSE_TRY.epw" "test_modified.idf"
"/Applications/EnergyPlus-8-8-0/energyplus-8.8.0" -a -x -r -w "GBR_Cardiff_CIBSE_TRY.epw" "test_basic_modified.idf"

# Below this = TESTING/NOTES

In [ ]:
# Load IDF into a list of lists (using basic text)

idf_filepath = "test.idf"

def isplit(iterable, splitters):
    return [list(g) for k,g in itertools.groupby(iterable,lambda x:x in splitters) if not k]

def nestrepl(lst, what, repl):
    for index, item in enumerate(lst):
        if type(item) == list:
            nestrepl(item, what, repl)
        else:
            if item == what:
                lst[index] = repl

with open(idf_filepath, "r") as f:
    content = isplit(isplit([re.sub("[ ]{2,}", "", re.sub("\r\n", "", re.sub("\t", "", x))) for x in f.readlines()], "")[0], "\n")
    content = [[x.replace('\n','') for x in l] for l in content]


* If length of sublist is 1 then only one value e.g. Timestep
* If sublist length of >1 then multiple values
* If count of sublist unique first values (if > 1) 


In [ ]:
Modify the following entries:


In [ ]:
# Remove unecessary parts
a = []
for i in content:
    if i[0] not in ["SizingPeriod:DesignDay,"]:
        a.append(i)
    else:
        pass
a

In [ ]:
[i[0] for i in content]